# Manipulate data the MXNet way with `ndarray`

It's impossible to get anything done if we can't manipulate data. So let's start by introducing NDArrays, MXNet's primary tool for storing and transforming data. If you've worked with NumPy before, you'll notice that NDArrays are by design similar to NumPy's multi-dimensional array. However, they confer a few key advantages. First, NDArrays support asynchronous computation on CPU, GPU, and distributed cloud architectures. Second, they provide support for automatic differentiation. These properties make NDArray an ideal library for machine learning, both for researchers and engineers launching production systems.


## Getting started

In this chapter, we'll get you going with the basic functionality. Don't worry if you don't understand any of the basic math, like element-wise operations or normal distributions. In the next two chapters we'll take another pass at NDArray, teaching you both the math you'll need and how to realize it in code.

To get started, let's import `mxnet`. We'll also `ndarray` from `mxnet` for convenience. We’ll also make a habit of setting a random seed so that you always get the same results that we do.

In [2]:
import mxnet as mx
from mxnet import nd
mx.random.seed(1)

Next, let's see how to create an NDArray, without values initialized to 1. Specifically, we'll create a 2D array (also called a *matrix*) with 3 rows and 4 columns.

In [4]:
x = nd.empty(shape=(3,4))
print(x)


[[  5.30887333e+25   4.58644987e-41   8.72613889e-38   0.00000000e+00]
 [  2.10587223e-37   0.00000000e+00   8.40779079e-45   0.00000000e+00]
 [  0.00000000e+00   1.26116862e-44   1.40129846e-44   4.58644987e-41]]
<NDArray 3x4 @cpu(0)>


The `empty` method just grabs some memory and hands us back a matrix without setting the values of any of its entries. But typically, we'll want our matrices initialized. Commonly, we want a matrix of all zeros. 

In [48]:
x = nd.zeros(shape=(3,5))
x


[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]
<NDArray 3x5 @cpu(0)>

Similarly, `ndarray` has a function to create a matrix of all ones. 

In [49]:
x = nd.ones(shape=(3,4))
x


[[ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]]
<NDArray 3x4 @cpu(0)>

Often, we'll want to create arrays whose values are sampled randomly. This is especially common when we intend to use the array as a parameter in a neural network. In this snippet, we initialize with values drawn from a standard normal distribution.

In [50]:
y = nd.random_normal(shape=(3,4))
y


[[ 0.14867957 -0.21776067 -0.49851316  2.22578788]
 [-0.84815776 -0.68832648  0.07811152  0.84155577]
 [-0.38241303 -0.61584121 -0.05334064  0.01961165]]
<NDArray 3x4 @cpu(0)>

As in NumPy, the dimensions of each NDArray are accessible via the `.shape` attribute.

In [51]:
y.shape

(3, 4)

We can also query its size, which is equal to the product of the components of the shape. Together with the precision of the stored values, this tells us how much memory the array occupies.

In [52]:
y.size

12

## Operations

NDarray supports a large number of standard mathematical operations. Such as element-wise addition:

In [53]:
x + y


[[ 1.14867961  0.78223932  0.50148684  3.22578788]
 [ 0.15184224  0.31167352  1.07811153  1.84155583]
 [ 0.61758697  0.38415879  0.94665939  1.0196116 ]]
<NDArray 3x4 @cpu(0)>

Multiplication:

In [54]:
x * y


[[ 0.14867957 -0.21776067 -0.49851316  2.22578788]
 [-0.84815776 -0.68832648  0.07811152  0.84155577]
 [-0.38241303 -0.61584121 -0.05334064  0.01961165]]
<NDArray 3x4 @cpu(0)>

In [55]:
nd.exp(y)


[[ 1.16030109  0.80431789  0.60743314  9.26077652]
 [ 0.42820305  0.50241619  1.08124328  2.31997347]
 [ 0.68221325  0.54018629  0.948057    1.01980519]]
<NDArray 3x4 @cpu(0)>

We can also grab a matrix's transpose compute a proper matrix-matrix product.

In [56]:
nd.dot(x, y.T)


[[ 1.65819359 -0.61681694 -1.03198326]
 [ 1.65819359 -0.61681694 -1.03198326]
 [ 1.65819359 -0.61681694 -1.03198326]]
<NDArray 3x3 @cpu(0)>

We'll explain these opoerations and present even more operators in the [linear algebra](P01-C03-linear-algebra.ipynb) chapter. But for now, we'll stick with the mechanics of working with NDArrays.

## In-place operations

In the previous example, every time we ran an operation, we allocated new memory to host its results. For example, if we write `y = x + y`, we will dereference the matrix that `y` used to point to and insted point it at the newly allocated memory. We can show this using Python's `id()` function, which tells us precisely which object a variable refers to.

In [57]:
print('id(y):', id(y))
y = y + x
print('id(y):', id(y))

id(y): 140571994896256
id(y): 140571994894576


To make better use of memory, we can perform operations in place, reusing already allocated memory. We can specify where to write the results of operations by assigning them with slice notation, e.g., `result[:] = ...`.

In [58]:
z = nd.zeros_like(x)
print('id(z):', id(z))
z[:] = x + y
print('id(z):', id(z))

id(z): 140572040919864
id(z): 140572040919864


If we're not planning to re-use ``x``, then we can assign the result to ``x`` itself. There are two ways to do this in MXNet. 
1. By using slice notation x[:] = x op y
2. By using the op-equals operators like `+=`

x += y
x

But be careful! This is NOT the same as x = x + y. If we don’t use slice notation then we allocate new memory and assign a reference to the new data to the variable x.

## Slicing

MXNet NDArrays support slicing in all the ridiculous ways you might imagine accessing your data. Here's an example of reading the second and third rows from ``x``.

In [59]:
x[1:2]


[[ 1.  1.  1.  1.]]
<NDArray 1x4 @cpu(0)>

Now let's try writing to a specific element.

In [60]:
x[1,2] = 9.0
x


[[ 1.  1.  1.  1.]
 [ 1.  1.  9.  1.]
 [ 1.  1.  1.  1.]]
<NDArray 3x4 @cpu(0)>

Multi-dimensional slicing is also supported.

In [61]:
x[1:2,1:3]


[[ 1.  9.]]
<NDArray 1x2 @cpu(0)>

In [62]:
x[1:2,1:3] = 5.0
x


[[ 1.  1.  1.  1.]
 [ 1.  5.  5.  1.]
 [ 1.  1.  1.  1.]]
<NDArray 3x4 @cpu(0)>

## Broadcasting

You might wonder, what happens if you add a vector ``y`` to a matrix ``X``? These operations, where we compose a low dimensional array ``y`` with a high-dimensional array ``X`` invoke a functionality called broadcasting. Here, the low-dimensional array is duplicated along any axis with dimension ``1`` to match the shape of the high dimesnional array. Consider the following example.

In [63]:
x = nd.ones(shape=(3,3))
print('x = ', x)
y = nd.arange(3)
print('y = ', y)
print('x + y = ', x + y)

x =  
[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]
<NDArray 3x3 @cpu(0)>
y =  
[ 0.  1.  2.]
<NDArray 3 @cpu(0)>
x + y =  
[[ 1.  2.  3.]
 [ 1.  2.  3.]
 [ 1.  2.  3.]]
<NDArray 3x3 @cpu(0)>


While `y` is initially of shape (3), MXNet infers its shape to be (1,3), and then broadcasts along the rows to form a (3,3) matrix). You might wonder, why did MXNet choose to interpret `y` as a (1,3) matrix and not (3,1). That's because broadcasting prefers to duplicate along the left most axis. We can alter this behavior by explicitly giving `y` a 2D shape.

In [65]:
y = y.reshape((3,1))
print('y = ', y)
print('x + y = ', x+y)

y =  
[[ 0.]
 [ 1.]
 [ 2.]]
<NDArray 3x1 @cpu(0)>
x + y =  
[[ 1.  1.  1.]
 [ 2.  2.  2.]
 [ 3.  3.  3.]]
<NDArray 3x3 @cpu(0)>


## Converting from MXNet NDArray to NumPy 

Converting MXNet NDArrays to and from NumPy is easy. The converted arrays do not share memory.

In [66]:
a = x.asnumpy()
type(a)

numpy.ndarray

In [67]:
y = nd.array(a) 
y


[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]
<NDArray 3x3 @cpu(0)>

## Managing context

By far, MXNet NDArray looks almost identical to NumPy. One of the key features make MXNet differs to NumPy is the supporting for various hardware.

In MXNet, every array has a context. One context could be the CPU. Other contexts might be various GPUs. Things can get even hairier when we deploy jobs across multiple servers. By assigning arrays to contexts intelligently, we can minimize the time spent transferring data between devices. For example, when training neural networks on a server with a GPU, we typically prefer for the model's parameters to live on the GPU. To start, let's try initializing an array on the first GPU.

In [71]:
from mxnet import gpu
z = nd.ones(shape=(3,3), ctx=gpu(0))
z


[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]
<NDArray 3x3 @gpu(0)>

Given an NDArray on a given context, we can copy it to another context by using the ``copyto()`` method.

In [72]:
x_gpu = x.copyto(gpu(0))
print(x_gpu)


[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]
<NDArray 3x3 @gpu(0)>


The result of an operator will have the same context as the inputs.

In [73]:
x_gpu + z


[[ 2.  2.  2.]
 [ 2.  2.  2.]
 [ 2.  2.  2.]]
<NDArray 3x3 @gpu(0)>

## Watch out!

Imagine that your variable ``z`` already lives on your second GPU (``gpu(0)``). What happens if we call ``z.copyto(gpu(0))``? It will make a copy and allocate new memory, even though that variable already lives on the desired device! 

Often, we only want to make a copy if the variable *currently* lives in the wrong context. In these cases, we can call ``as_in_context()``. If the variable is already on ``gpu(0)`` then this is a no-op.

In [74]:
print('id(z):', id(z))
z = z.copyto(gpu(0))
print('id(z):', id(z))
z = z.as_in_context(gpu(0))
print('id(z):', id(z))
print(z)

id(z): 140571939657432
id(z): 140572040919864
id(z): 140572040919864

[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]
<NDArray 3x3 @gpu(0)>


For whinges or inquiries, [open an issue on  GitHub.](https://github.com/zackchase/mxnet-the-straight-dope)